# HW7 DS610

Streaming Data

In this homework you will be running the code below to demonstrate a basic example of perform a streamed query.

There are two files attached;
* logfile1.json
* logfile2.json

Put this jupyter notebook into a local EMTPY folder on your computer. Be sure that java is installed on your computer!
* If you dont have it, download it from here; https://www.oracle.com/java/technologies/javase-downloads.html
* Check your terminal by typing "java -version" to see if you have it downloaded

Once you have this notebook in an EMPTY folder, then one at a time, add logfile1.json and logfile2.json to the folder AFTER running the script below.

In [ ]:
try:
    from pyspark.sql import SparkSession
except:
    !pip install pyspark
    from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType

import os
current_dir = os.path.abspath(os.curdir)

schema = '''
StructType([StructField('Arrival_Time', LongType(), True), 
StructField('Creation_Time', LongType(), True), StructField('Device', StringType(), True),
StructField('Index', LongType(), True), StructField('Model', StringType(), True),
StructField('User', StringType(), True), StructField('gt', StringType(), True),
StructField('x', DoubleType(), True), StructField('y', DoubleType(), True), 
StructField('z', DoubleType(), True)])
'''

spark = SparkSession.builder.config("HW7", current_dir).appName("StructuredStreaming").getOrCreate()
streaming = spark.readStream.schema(eval(schema)).json(current_dir)
# Run forever until terminated

stream = streaming.withColumn('Arrival_Time',f.to_timestamp((f.col('Arrival_Time')/1000)))\
                  .withColumn('Creation_Time',f.to_timestamp((f.col('Creation_Time')/1000000000)))\
                  .groupby('User')\
                  .agg(f.max(f.col('Arrival_Time')),f.min(f.col('Arrival_Time')))

query = stream.writeStream.outputMode("complete").format("console").queryName("arrivals").start()

query.awaitTermination()

# Cleanly shut down the session
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/07 19:15:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/07 19:15:06 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/hj/xyjyq1ds11n1ycjbp53_4svw0000gp/T/temporary-a8f204f6-9078-4bb5-86c9-149321b1e59f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/07 19:15:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----------------+-----------------+
|User|max(Arrival_Time)|min(Arrival_Time)|
+----+-----------------+-----------------+
|null|             null|             null|
+----+-----------------+-----------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+--------------------+
|User|   max(Arrival_Time)|   min(Arrival_Time)|
+----+--------------------+--------------------+
|   g|2015-02-23 05:57:...|2015-02-23 05:18:...|
|null|                null|                null|
|   f|2015-02-24 07:32:...|2015-02-24 06:56:...|
|   e|2015-02-24 09:51:...|2015-02-24 09:13:...|
|   h|2015-02-23 09:17:...|2015-02-23 08:45:...|
|   d|2015-02-24 08:11:...|2015-02-24 07:41:...|
|   c|2015-02-23 07:45:...|2015-02-23 07:12:...|
|   i|2015-02-24 06:49:...|2015-02-24 06:12:...|
|   b|2015-02-24 09:08:...|2015-02-24 08:30:...|
|   a|2015-02-23 08:35:...|2015-02-23 08:03:...|
+----+--------------------+--------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+--------------------+--------------------+
|User|   max(Arrival_Time)|   min(Arrival_Time)|
+----+--------------------+--------------------+
|   g|2015-02-23 05:57:...|2015-02-23 05:18:...|
|null|                null|                null|
|   f|2015-02-24 07:32:...|2015-02-24 06:56:...|
|   e|2015-02-24 09:51:...|2015-02-24 09:13:...|
|   h|2015-02-23 09:17:...|2015-02-23 08:45:...|
|   d|2015-02-24 08:11:...|2015-02-24 07:41:...|
|   c|2015-02-23 07:45:...|2015-02-23 07:12:...|
|   i|2015-02-24 06:49:...|2015-02-24 06:12:...|
|   b|2015-02-24 09:08:...|2015-02-24 08:30:...|
|   a|2015-02-23 08:35:...|2015-02-23 08:03:...|
+----+--------------------+--------------------+

